In [1]:
import pandas as pd

df = pd.read_csv('data/trends.csv')

# select distinct category from English speaking countries
df = df[df['location'].isin(['United Kingdom', 'Scotland', 'Australia', 'Ireland', 'New Zealand', 'Wales'])]
df = df[['category']].drop_duplicates()

# count the number of categories
df.size

147

In [2]:
df_wiki = pd.read_csv('data/children_cats.csv')
df_wiki = df_wiki.head(100000)

df_wiki['category'] = df_wiki.iloc[:, 0]
df_wiki = df_wiki[['category']].drop_duplicates()

df_wiki['category'] = df_wiki['category'].str.replace('_', ' ')

# shuffle
df_wiki = df_wiki.sample(frac=1).reset_index(drop=True)

# count the number of categories
df_wiki.size


100000

In [3]:
# TODO: Too long to run, also do we need it?
from transformers import BertTokenizer, BertModel
import torch  
from tqdm import tqdm  

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

import numpy as np  # Ensure numpy is imported

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

batch_size = 1024  # Set your desired batch size
mean_embeddings_np = np.empty((0, 768))  # Initialize an empty array for embeddings

# Process the data in batches
for i in tqdm(range(0, len(df_wiki), batch_size), desc="Processing batches"):  # Wrap the range with tqdm
    batch_df = df_wiki.iloc[i:i + batch_size]

    # Tokenize and encode the texts
    tokens = tokenizer(batch_df['category'].tolist(), return_tensors='pt', padding=True, truncation=True, max_length=512)
    tokens = {key: val.to(device) for key, val in tokens.items()}

    # Forward pass, get hidden states
    with torch.no_grad():
        outputs = model(**tokens)

    # Take the output embeddings from the last layer
    last_hidden_states = outputs.last_hidden_state

    # Pooling strategy: Take the mean of all token embeddings
    mean_embeddings = torch.mean(last_hidden_states, dim=1)

    # Convert the tensor to a numpy array and append to the mean_embeddings_np
    mean_embeddings_np = np.append(mean_embeddings_np, mean_embeddings.cpu().numpy(), axis=0)

# Assign the embeddings back to the DataFrame
df_wiki['embedding'] = list(mean_embeddings_np)

/home/ebk/anaconda3/envs/irs_proj/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ebk/anaconda3/envs/irs_proj/lib/python3.11/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647348947/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
Processing batches: 100%|███████████████████████| 98/98 [09:57<00:00,  6.09s/it]


In [4]:
df

,category
155,Men
160,Searches
165,Women
205,Popular Movies
210,Popular Queries
...,...
26640,DIY
26645,Fitness
26655,Lyrics
26690,When...?


In [5]:
df_wiki.to_csv('data/wiki_categories.csv_100k', index=False)

In [ ]:
df.to_csv('data/google_trends.csv', index=False)